In [1]:
import sys,os,time
import pandas as pd
import MySQLdb

In [10]:
day = time.strftime("%d_%m_%y")
connector = MySQLdb.connect(user='tcrd',db='tcrd600',host='tcrd.kmc.io')

In [11]:
tcrd_id = """SELECT id as tcrd_id,uniprot as Target_id FROM protein"""
df_id = pd.read_sql(tcrd_id,con=connector)
df_id.to_json('json_files/tcrd_id_'+day+'.json')

In [12]:
tcrd_target = """SELECT id as tcrd_id, tdl as Pharos_class, fam as protein_family, famext as protein_family_detail FROM target"""
df_target = pd.read_sql(tcrd_target,con=connector)
df_target.to_json('json_files/tcrd_target_'+day+'.json')

In [13]:
tcrd_info = """SELECT * FROM tdl_info """
df_info = pd.read_sql(tcrd_info,con=connector)
col_to_keep = ['Ab Count', 'MAb Count', 'NCBI Gene PubMed Count', 'EBI Total Patent Count', 'PubTator Score',
               'JensenLab PubMed Score']
df_info = df_info[df_info['itype'].isin(col_to_keep)]
df_info.loc[:,'value'] = df_info['number_value'].fillna(0) + df_info['integer_value'].fillna(0)
df_info = df_info.pivot(index='protein_id', columns='itype', values='value')
for col in col_to_keep:
    if col not in df_info.columns:
        df_info[col] = 0

df_info.index = df_info.index.astype(int)
df_info.reset_index(inplace=True)
df_info.to_json('json_files/tcrd_info_'+day+'.json')

In [14]:
tcrd_patent = """SELECT * FROM patent_count"""
df_patent = pd.read_sql(tcrd_patent,con=connector)
df_patent.drop(columns=['id'],inplace=True)
df_patent.to_json('json_files/tcrd_patent_'+day+'.json')

In [15]:
tcrd_disease = """SELECT TI.protein_id,TI.disease_id,T.doid,TI.score,T.name,DP.parent
FROM
tinx_importance TI
LEFT JOIN tinx_disease T ON TI.disease_id = T.id
LEFT JOIN do_parent DP on T.doid = DP.doid
ORDER BY TI.score DESC"""
df_disease = pd.read_sql(tcrd_disease,con=connector)
df_disease = df_disease[df_disease.score>1]
df_disease.to_json('json_files/tcrd_disease_'+day+'.json')

DatabaseError: Execution failed on sql 'SELECT TI.protein_id,TI.disease_id,T.doid,TI.score,T.name,DP.parent
FROM
tinx_importance TI
LEFT JOIN tinx_disease T ON TI.disease_id = T.id
LEFT JOIN do_parent DP on T.doid = DP.doid
ORDER BY TI.score DESC': (1054, "Unknown column 'DP.parent' in 'field list'")

In [16]:
tcrd_novelty = """SELECT score,protein_id FROM tinx_novelty"""
df_novelty = pd.read_sql(tcrd_novelty,con=connector)
df_novelty.to_json('json_files/tcrd_novelty_'+day+'.json')